In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer,KNNImputer

In [2]:
df_train = pd.read_csv("data/train.csv", index_col='PassengerId')
df_test = pd.read_csv("data/test.csv", index_col='PassengerId')
df_train.head(2)

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
PassengerId,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True


# Data Treatment

In [3]:
def feature_inputer(
    df:pd.DataFrame,
    to_mode = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP'],
    to_median = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
) -> pd.DataFrame:
    for col in to_mode:
        mode_inputer = SimpleImputer(strategy='most_frequent')
        df[col] = mode_inputer.fit_transform(df[[col]])

    
    for col in to_median:
        inputer = SimpleImputer(strategy='median')
        df[col] = inputer.fit_transform(df[[col]])
    return df

def dtype_memory_reducer(df: pd.DataFrame) -> pd.DataFrame:
    df['CryoSleep'] = df['CryoSleep'].astype(bool)
    df['VIP'] = df['VIP'].astype(bool)

    df['HomePlanet'] = df['HomePlanet'].astype('category')
    df['Destination'] = df['Destination'].astype('category')
    return df

def outliers_to_log(df:pd.DataFrame) -> pd.DataFrame:
    to_log = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    for col in to_log:
        df[col] = np.log(df[col] + 1)
    return df

def cabin_inputer(df:pd.DataFrame) -> pd.DataFrame:
    cabin_features = df['Cabin'].str.split("/",expand=True)[[0,2]].rename(columns={0:'Deck',2:'side'})
    df = pd.concat([df,cabin_features],axis=1)
    df['Deck'] = df['Deck'].fillna('G').replace("T","G")
    df = df.drop(['side','Cabin'],axis=1)
    return df

def vip_knn_input(df: pd.DataFrame) -> pd.DataFrame:
    inputer = KNNImputer(n_neighbors=5)
    df['VIP'] = inputer.fit_transform(df[['VIP','RoomService']])[:,0]
    return df

def treat_dataset(df:pd.DataFrame) -> pd.DataFrame:
    df = df.drop(['Name'],axis=1)
    df = cabin_inputer(df)
    df = feature_inputer(df)
    df = vip_knn_input(df)
    df['0_bills'] = df[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1) == 0
    df = outliers_to_log(df)
    df = dtype_memory_reducer(df)
    return df

def feature_enginnering(df: pd.DataFrame) -> pd.DataFrame:
    df = pd.get_dummies(df,columns=['HomePlanet','Destination','Deck'],drop_first=True)
    return df

In [4]:
df_train = treat_dataset(df_train.copy())
df_train = feature_enginnering(df_train)

In [5]:
X_train,y_train = df_train.drop('Transported',axis=1), df_train['Transported']

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

## Cross Validation

In [7]:
from sklearn.model_selection import cross_val_score, KFold

def model_evaluation(model,x_train,y_train):
    model.fit(x_train, y_train)
    scores = cross_val_score(model, x_train, y_train, cv=KFold(5, shuffle=True, random_state=42), scoring = "accuracy")
    return {
        'model':model,
        'score_avg':np.mean(scores),
        'score_std':np.std(scores)
    }

In [22]:
models = [
    LogisticRegression(max_iter=1000),
    RandomForestClassifier(min_samples_leaf=30,random_state=42),
    KNeighborsClassifier(n_neighbors=10),
    MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
    
]

In [23]:
df_eva = [model_evaluation(model,X_train,y_train) for model in models]
pd.DataFrame(df_eva)

c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptr

,model,score_avg,score_std
0,LogisticRegression(max_iter=1000),0.771312,0.008187
1,"(DecisionTreeClassifier(max_features='sqrt', m...",0.792592,0.005461
2,KNeighborsClassifier(n_neighbors=10),0.764639,0.005129
3,"MLPClassifier(alpha=1e-05, hidden_layer_sizes=...",0.773958,0.009612


# Remodelling with Bagging

In [24]:
X_train_bagging = {
'Logistic Regression':df_eva[0]['model'].predict_proba(X_train)[:,1],
'Decision Tree':df_eva[1]['model'].predict_proba(X_train)[:,1],
'KNN':df_eva[2]['model'].predict_proba(X_train)[:,1],
'MLP':df_eva[3]['model'].predict_proba(X_train)[:,1],
}
X_train_bagging = pd.DataFrame(X_train_bagging, X_train.index)

In [27]:
bagging_model = RandomForestClassifier(min_samples_leaf=30,random_state=42)
eva_bagging = model_evaluation(bagging_model,X_train_bagging,y_train)
eva_bagging

{'model': RandomForestClassifier(min_samples_leaf=30, random_state=42),
 'score_avg': 0.818589774555301,
 'score_std': 0.006478447503535945}

# Submission

In [28]:
X_test = treat_dataset(df_test)
X_test = feature_enginnering(X_test)
X_test.head(2)

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,0_bills,HomePlanet_Europa,HomePlanet_Mars,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G
PassengerId,,,,,,,,,,,,,,,,,,,
0013_01,True,27.0,False,0.0,0.000000,0.0,0.00000,0.0,True,0,0,0,1,0,0,0,0,0,1
0018_01,False,19.0,False,0.0,2.302585,0.0,7.94591,0.0,False,0,0,0,1,0,0,0,0,1,0


In [29]:
X_test_bagging = {
'Logistic Regression':df_eva[0]['model'].predict_proba(X_test)[:,1],
'Decision Tree':df_eva[1]['model'].predict_proba(X_test)[:,1]
}
X_test_bagging = pd.DataFrame(X_test_bagging, X_test.index)
X_test_bagging.head(2)

,Logistic Regression,Decision Tree
PassengerId,,
0013_01,0.706534,0.631579
0018_01,0.190066,0.000000


In [30]:
y_test = bagging_model.predict(X_test_bagging)

In [31]:
submission = pd.Series(y_test, index=[X_test.index]).to_frame('Transported')
submission.to_csv('data/submission_bagging_2022-10-29.csv')